In [5]:
# Install CUDA C++ plugin for Colab:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!apt update
!apt install -y libopencv-dev pkg-config


The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,227 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-dri

In [10]:
%%writefile cpu_gaussian_blur.cpp
#include <iostream>
#include <opencv2/opencv.hpp>
#include <chrono>

using namespace std;
using namespace cv;
using namespace std::chrono;

// 3x3 Gaussian Blur on CPU
void gaussianBlurCPU(const Mat &input, Mat &output) {
    int kernel[3][3] = {
        {1, 2, 1},
        {2, 4, 2},
        {1, 2, 1}
    };
    int kernelSum = 16;

    int height = input.rows;
    int width  = input.cols;

    input.copyTo(output);

    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {

            int sum = 0;

            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    int pixel = input.at<uchar>(y + ky, x + kx);
                    sum += pixel * kernel[ky + 1][kx + 1];
                }
            }

            output.at<uchar>(y, x) = sum / kernelSum;
        }
    }
}

int main() {
    Mat inputColor = imread("/content/download.jpg");
    if (inputColor.empty()) {
        cout << "❌ Error: Image not found!" << endl;
        return -1;
    }

    Mat inputGray;
    cvtColor(inputColor, inputGray, COLOR_BGR2GRAY);

    Mat outputGray = Mat::zeros(inputGray.size(), inputGray.type());

    auto start = high_resolution_clock::now();
    gaussianBlurCPU(inputGray, outputGray);
    auto end = high_resolution_clock::now();

    auto duration = duration_cast<milliseconds>(end - start);
    cout << "✅ CPU Gaussian Blur Time: " << duration.count() << " ms" << endl;

    imwrite("input_gray.png", inputGray);
    imwrite("output_cpu_blur.png", outputGray);

    cout << "📁 Output saved as output_cpu_blur.png" << endl;
    return 0;
}


Overwriting cpu_gaussian_blur.cpp


In [11]:
!g++ cpu_gaussian_blur.cpp -o cpu_blur `pkg-config --cflags --libs opencv4`


In [12]:
!./cpu_blur


✅ CPU Gaussian Blur Time: 4 ms
📁 Output saved as output_cpu_blur.png


In [19]:
%%writefile gpu_gaussian_blur_opencv.cu
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>
#include <stdio.h>

// 3x3 Gaussian Blur Kernel
__global__ void gaussianBlurKernel(
    unsigned char* input,
    unsigned char* output,
    int width,
    int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x > 0 && x < width - 1 && y > 0 && y < height - 1) {
        int kernel[3][3] = {
            {1, 2, 1},
            {2, 4, 2},
            {1, 2, 1}
        };

        int sum = 0;
        for (int ky = -1; ky <= 1; ky++) {
            for (int kx = -1; kx <= 1; kx++) {
                int pixel = input[(y + ky) * width + (x + kx)];
                sum += pixel * kernel[ky + 1][kx + 1];
            }
        }
        output[y * width + x] = sum / 16;
    }
}

int main() {
    cv::Mat inputColor = cv::imread("/content/download.jpg");
    if (inputColor.empty()) {
        printf("Image not found\n");
        return -1;
    }

    cv::Mat inputGray;
    cv::cvtColor(inputColor, inputGray, cv::COLOR_BGR2GRAY);

    int width = inputGray.cols;
    int height = inputGray.rows;
    size_t size = width * height;

    cv::Mat outputGray(height, width, CV_8UC1);

    unsigned char *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);

    cudaMemcpy(d_input, inputGray.data, size, cudaMemcpyHostToDevice);

    dim3 block(16,16);
    dim3 grid((width+15)/16, (height+15)/16);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    gaussianBlurKernel<<<grid, block>>>(d_input, d_output, width, height);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float gpuTime;
    cudaEventElapsedTime(&gpuTime, start, stop);

    cudaMemcpy(outputGray.data, d_output, size, cudaMemcpyDeviceToHost);

    cv::imwrite("output_gpu_blur.png", outputGray);

    printf("GPU Gaussian Blur Time: %.3f ms\n", gpuTime);

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Writing gpu_gaussian_blur_opencv.cu


In [20]:
!nvcc gpu_gaussian_blur_opencv.cu -o gpu_blur \
  `pkg-config --cflags --libs opencv4` \
  -arch=sm_75


/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [21]:
!./gpu_blur


GPU Gaussian Blur Time: 0.101 ms
